# QComp - Coding Session #4: Oracle Synthesis

This is the fourth part of a series of lab sessions to implement and analyze the HHL algorithm.

* In Coding Session #1, we implemented QPE
* In Coding Session #2, we implemented HHL
* In Coding Session #3, we saw a possible strategy to implement $e^{itA}$
* In this coding session, we will discuss how to implement an oracle such as the inversion oracle of HHL.

This session is only for practice.

First, make sure libraries are installed (you can uncomment those if needed)

In [1]:
! python -m pip install matplotlib sympy scipy
! python -m pip install qiskit qiskit-aer

Now, some libraries to load (nothing to modify here)

In [2]:
# Set-up for long lines
from IPython import get_ipython
from IPython.display import display, HTML

if get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
    print("Running inside Jupyter")
    display(HTML("<style>pre { white-space: pre !important; }</style>"))

# Bunch of imports
import itertools, math, random
import sympy, scipy
import numpy as np
import matplotlib.pyplot as plt
from math import pi, gcd
from qiskit import *
from qiskit.circuit import *
from qiskit.circuit.library import *
from qiskit.compiler import transpile
from qiskit.quantum_info.operators import Operator
from qiskit_aer import AerSimulator, StatevectorSimulator, UnitarySimulator
from scipy import linalg, optimize
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']

Running inside Jupyter


Small library for pretty-printing (nothing to do)

In [3]:
def nat2bl(pad,n):
    assert(n >= 0)
    if n == 0: r = [0 for i in range(pad)]
    elif n % 2 == 1: r = nat2bl(pad-1,(n-1)//2); r.append(1)
    else: r = nat2bl(pad-1,n//2); r.append(0)
    return r

def bl2nat(s):
    s = [] + s # Make sure to not change s
    if len(s) == 0: return 0
    else: a = s.pop(); return (a + 2*bl2nat(s))

def bl2bs(l):
    if len(l) == 0: return ""
    else: a = l.pop(); return (bl2bs(l) + str(a))

def nat2bs(pad,i): return bl2bs(nat2bl(pad,i))

def bs2bl(s):
    l = []
    for i in range(len(s)): l.append(int(s[i]))
    return l

def bs2nat(s): return bl2nat(bs2bl(s))

# 0 - Our very own notion of circuits

In this lab session, we discuss reversible circuits: circuits built
from X, NOT and Toffoli gates. As these are bijections on basis ket
vectors, simulating a reversible circuit only require Boolean values:
it should be doable efficiently.

Unfortunately, Qiskit does not come (afaik) with a simulation backend
for such reversible circuits. We therefore have to build our very own.

For the sake of simplicity, we here define a very simple class for
circuits.

In [4]:

from functools import reduce

class RevCirc:
    def __init__(self,inputs):
        self.gates = []
        self.ancillas = set()
        self.inputs = inputs
        self.wires = set(inputs)
        self.ctl = []
    
    def x(self,i):       self.gates.append({"target" : i, "ctl" : [] + self.ctl})
    def cx(self,j,i):    self.gates.append({"target" : i, "ctl" : [j] + self.ctl})
    def ccx(self,j,k,i): self.gates.append({"target" : i, "ctl" : [j,k] + self.ctl})

    def alloc(self):
        w = max(list(self.wires) + [0]) + 1
        self.ancillas.add(w)
        self.wires.add(w)
        return w
    
    def allocn(self,n):
        return [self.alloc() for _ in range(n)]

    def append(self, circ, inputs):
        assert(len(circ.inputs) == len(inputs))
        ancillas = self.allocn(len(circ.ancillas))
        renaming = dict(zip(circ.inputs + list(circ.ancillas), inputs + ancillas))
        for g in circ.gates:
            self.gates.append({"target" : renaming[g["target"]],
                               "ctl" : [renaming[c] for c in g["ctl"]] + self.ctl})
        return renaming
        
    def run(self,state):
        assert(set(self.inputs) == set(state.keys()))
        for a in self.ancillas: state[a] = 0
        for g in self.gates: state[g["target"]] ^= reduce(lambda x, y: x & y, [1] + [state[i] for i in g["ctl"]])
        return state

    def draw(self):
        height = max(self.wires)
        res_string = ""
        def isIn(wn, gn): return wn < max(gn["ctl"] + [gn["target"]]) and wn >= min(gn["ctl"] + [gn["target"]])
        for wn in sorted(list(self.wires)):
            if wn in self.ancillas:  res_string += "a" + str(wn) + (" " * (len(str(height)) - len(str(wn)))) + " "
            else: res_string += "i" + str(wn) + (" " * (len(str(height)) - len(str(wn)))) + " "
            for gn in self.gates:
                if gn["target"] == wn: res_string += "--X--"
                elif wn in gn["ctl"]:  res_string += "--.--"
                elif isIn(wn,gn):      res_string += "--|--"
                else:                  res_string += "-----"
            res_string += "\n  " + (" " * (len(str(height))))
            for gn in self.gates:
                if isIn(wn,gn):  res_string += "  |  "
                else:            res_string += "     "
            res_string += "\n"
        print(res_string)

def getValue(valuation, register):
    return [valuation[wire] for wire in register]


Reversible circuits are made of wires, here simply natural
numbers. There are two types of wires: inputs wires, set at init time,
and ancillas that can be added on the fly. Wires cannot be erased:
they are all accessible at the end of the circuit through their ID
number. The operations on circuits are
 * `circ.x(q)` : apply an X gate on `q`
 * `circ.cx(q1,q2)` : apply a CNOT on `q2`, the control is `q1`
 * `circ.ccx(q1,q2,q3)~ : apply a Toffoli gate on `q3`, the controls are `q1` and `q2`
 * `q = circ.alloc()` : allocate a new ancilla `q` (assumed initialized at 0)
 * `q = circ.allocn(n)` : allocate `n` new ancillas and store them in list `q`  (they are assumed initialized at 0)
 * `circ.draw()` : pretty-print the circuit
 * `valuation = circ.run(input_dict)` : run the circuit with a dictionary of the form `{q1 : 0, q2: 1, ...}` where `q1`, `q2`, etc are the input qubits. It returns a dictionary `valuation` stating the value of each wire of the circuit at the end of the computation.

The function `getValue(valuation, register)` is a wrapper to get the bitstring value of a register, given a valuation coming from runnning a circuit.

A small example of circuit is as follows.

In [5]:

def example():
    circ = RevCirc([1,2])
    
    circ.x(1)
    q = circ.alloc()
    circ.cx(1,q)
    circ.ccx(1,2,q)
    
    circ.draw()

    input_val = dict(zip([1,2], [0,0])) # wires and 2 are set to 0
    print("input valuation:" , input_val)
    
    val = circ.run(input_val)
    
    print("output valuation:", val)
    print(f"The register {str([1,2,q])} is in state", getValue(val, [1,2,q]))

example()

i1 --X----.----.--
          |    |  
i2 -------|----.--
          |    |  
a3 -------X----X--
                  

input valuation: {1: 0, 2: 0}
output valuation: {1: 1, 2: 0, 3: 1}
The register [1, 2, 3] is in state [1, 0, 1]



# 1 - A Ripple-Carry Adder

In this section, we will define the $V$-version of an adder: a circuit of shape

$$
V_{\text{adder}} : |{a}\rangle_n\otimes|{b}\rangle_n\otimes|{0\ldots0}\rangle_N\otimes|{0}\rangle_1\otimes|{0}\rangle_n
\mapsto
|{a}\rangle_n\otimes|{b}\rangle_n\otimes|{\text{garbage}}\rangle_N\otimes|{\text{carry}}\rangle_1\otimes|{a+b}\rangle_n
$$

The notation $|{-}\rangle_k$ means that the register is of size $k$. In
particular, our adder stores the last carry coming from the addition
(if any) in an auxiliary register.

## 1.1 - Building-Blocks

Remember the building blocks HA (half-adder) and FA (full-adder) for
an adder, as discussed in Section 5.3.9 of the lecture notes (and on
the board during last lecture).

**TODO** Implement HA and FA with the following specification:

* `s,c = HA(circ, a, b)` inputs a circuit `circ`, two existing wires `a` and `b` of `circ`. It creates two ancillas `s` and `c` and sets `s` to $\texttt{a} \oplus\texttt{b}$ and `c` to $\texttt{a}\texttt{b}$.
* `s,c = FA(circ, a, b, c)` has a similar structure, but it sets `s` and `c` to

$$
s = a\oplus b\oplus c
\qquad
c = ab\oplus ac\oplus bc
$$

In [6]:

def HA(circ, a, b):
    s = circ.alloc()
    c = circ.alloc()
    ## TODO BEGIN
    circ.cx(a, s)
    circ.cx(b, s)
    circ.ccx(a, b, c)


    ## TODO END
    return (s,c)

def FA(circ, a, b, cin):
# def FA(circ, cin, a, b):
    s = circ.alloc()
    cout = circ.alloc()
    ## TODO BEGIN
    circ.cx(a, s)
    circ.cx(b, s)
    circ.cx(cin, s)

    circ.ccx(a, b, cout)
    circ.ccx(a, cin, cout)
    circ.ccx(b, cin, cout)



    ## TODO END
    return (s,cout)


## Printing the circuit, to check

def test_HA_FA():
    print("Half-adder")
    circ = RevCirc([0,1])
    HA(circ,0,1)
    circ.draw()
    
    print("Full-adder")
    circ = RevCirc([0,1,2])
    FA(circ,0,1,2)
    circ.draw()

test_HA_FA()

Half-adder
i0 --.---------.--
     |         |  
i1 --|----.----.--
     |    |    |  
a2 --X----X----|--
               |  
a3 ------------X--
                  

Full-adder
i0 --.--------------.----.-------
     |              |    |       
i1 --|----.---------.----|----.--
     |    |         |    |    |  
i2 --|----|----.----|----.----.--
     |    |    |    |    |    |  
a3 --X----X----X----|----|----|--
                    |    |    |  
a4 -----------------X----X----X--
                                 




## 1.2 - Composing the Pieces Together

The complete adder (in $V$-form) is one half-adder followed by a chain
of full-adder, each one sending its carry to the next full adder. The
last carry is left in its own register.

We need to choose a binary encoding for natural numbers: let us set
the least significant bit on the right. This is the convention used in
`nat2bl` and `bl2nat`.

**TODO** Implement the $V$-form of the adder.

In [ ]:

def adderV(circ, aa, bb): # assume aa and bb have same size
    res = []          ## TO UPDATE
    c = circ.alloc()  ## TO UPDATE
    N = len(aa)

    ## TODO BEGIN
    # First, the Half adder on the least significant bit
    s, c = HA(circ, aa[N - 1], bb[N - 1])
    res.append(s)

    # Then, the Full adder on the reverse order from N - 2 to 0 
    for i in range(N - 2, -1,  -1):
        s, cout = FA(circ, aa[i], bb[i], c)
        res = [s] + res
        c = cout

    ## TODO END

    return res,c



Testing cell.

In [43]:

def testAdderV():
    # Size of the bitstring to encode numbers
    # Change to test
    N = 4
    
    # What to add together -- should fit on N bits
    # Change to test
    a = 10
    b = 3
    
    # Do not change the rest
    
    ar = list(range(0,N))
    br = list(range(N,2*N))
    
    circ = RevCirc(ar + br)
    s,c = adderV(circ,ar,br)
    
    circ.draw()
    
    ai = nat2bl(N,a)
    bi = nat2bl(N,b)
    
    print(f"{a} in binary is", ai)
    print(f"{b} in binary is", bi)
    
    d = dict(zip(ar + br, ai + bi))
    
    output = circ.run(d)
    si = getValue(output, s)
    
    si_str = str(si)
    s = bl2nat(si)
    print(f"The sum of {a} and {b} is {a+b}")
    print(f"The answer given by the circuit is {s}")
    print("In binary form:", si_str)
    print(f"The carry bit is {d[c]}")

testAdderV()



test
2
1
0
i0  -----------------------------------------------------------------------------.--------------.----.-------
                                                                                 |              |    |       
i1  -----------------------------------------------.--------------.----.---------|--------------|----|-------
                                                   |              |    |         |              |    |       
i2  -----------------.--------------.----.---------|--------------|----|---------|--------------|----|-------
                     |              |    |         |              |    |         |              |    |       
i3  --.---------.----|--------------|----|---------|--------------|----|---------|--------------|----|-------
      |         |    |              |    |         |              |    |         |              |    |       
i4  --|---------|----|--------------|----|---------|--------------|----|---------|----.---------.----|----.--


# 2 - Multiplication

Consider $a$ and $b$ two natural numbers coded in binary on $5$ bits.
The multiplication $a\cdot b$ of $a$ and $b$ fits on $10$ bits.  The
procedure is as you learned in elementary school.

Suppose that $a$ is $a_1a_2a_3a_4a_5$ and $b$ is $b_1b_2b_3b_4b_5$ in
binary, least significant bit on the right.
We can write $a\cdot b$ as follows:

$$
a \cdot b = b_1 \cdot 2^4 \cdot a + b_2 \cdot 2^3 \cdot a + b_3 \cdot 2^2 \cdot a +
b_4 \cdot 2^1 \cdot a + b_5 \cdot 2^0 \cdot a
$$

The multiplication of $a$ by $2^k$ corresponds to padding
$a_1a_2a_3a_4a_5$ with $0$'s on the left. One simple possibility for implementing $a\cdot b$ is therefore to consider a series of registers of size $10$ as follows:

$$
\begin{array}{lllllll lllll}
   b_5 & {\cdot} & 0 & 0 & 0 & 0 & 0 &  a_1 & a_2 & a_3 & a_4 & a_5 \\
   b_4 & {\cdot} & 0 & 0 & 0 & 0 & a_1 &  a_2 & a_3 & a_4 & a_5 & 0 \\
   b_3 & {\cdot} & 0 & 0 & 0 & a_1 & a_2 &  a_3 & a_4 & a_5 & 0 & 0 \\
   b_2 & {\cdot} & 0 & 0 & a_1 & a_2 & a_3 &  a_4 & a_5 & 0 & 0 & 0 \\
   b_1 & {\cdot} & 0 & a_1 & a_2 & a_3 & a_4 &  a_5 & 0 & 0 & 0 & 0 \\
\end{array}
$$

The operation $b_k\cdot \texttt[register]$ is to be understood as: leave the
corresponding register to $00000 00000$ when $b_k$ is $0$.

Once these registers are set up, it is just a matter of adding them
together with the function `adderV` you defined above.

**TODO** Implement the $V$-form of the multiplication of two registers
  of same size, as described in this section.

In [ ]:
# Multiplication

def mult(circ, aa, bb):
    # This is what I mean when I suggest to allocate auxiliary registers
    aux = [circ.allocn(len(aa) + len(bb)) for i in range(len(bb))]
    print(aux)

    # You now have to fill them with the correct bitstring, add them
    # together, and return the final register with the last carry

    ss = [] # Placeholder for the final register, to redefine correctly
    
    ## TODO BEGIN
    size_register = len(aa) + len(bb)
    for i in range(len(bb)):
        current_register = aux[i]
        




    ## TODO END
    
    return ss   # Return the final register


Testing cell.

In [48]:

def testMult():
    # Size of registers (to play with)
    N = 6

    # Values to multiply (make sure they fit in registers of size N)
    a = 33
    b = 48

    # Do not modify below
    
    ar = list(range(0,N))
    br = list(range(N,2*N))
    
    circ = RevCirc(ar + br)
    s = mult(circ,ar,br)
    
    ai = nat2bl(N,a)
    bi = nat2bl(N,b)
    
    d = dict(zip(ar + br, ai + bi))
    
    print("nber of gates:", len(circ.gates))
    output = circ.run(d)
    
    si = getValue(output, s)
    
    print(f"First value is {a}, the register is", ai)
    print(f"Second value is {b}, the register is", bi)
    print(f"The circuit outputs {str(si)}, meaning {bl2nat(si)}")
    print(f"The correct value should have been {a*b}, not counting the overflow")
    
testMult()



[[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23], [24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35], [36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47], [48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59], [60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71], [72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83]]
nber of gates: 0
First value is 33, the register is [1, 0, 0, 0, 0, 1]
Second value is 48, the register is [1, 1, 0, 0, 0, 0]
The circuit outputs [], meaning 0
The correct value should have been 1584, not counting the overflow



# 3 - Signed, fixed point representation of real numbers

The two functions you just wrote can be composed to generate
polynomials. An interesting class of polynomials is the class of
Taylor polynomials, for approximating functions such as $x \mapsto
arcsin(\frac{1}{16x}$.

However, such functions works over real numbers... Going from
bitstring representing natural numbers to bitstring representing real
numbers can be done in two steps:

1. Signed Representation. One consider the high bit as the bit
representing the sign. So for instance, signed integers on $3$ bits
are

$$
\begin{array}{lll}
  0 0 0 & \simeq & 0\\
  0 0 1 & \simeq & 1\\
  0 1 0 & \simeq & 2\\
  0 1 1 & \simeq & 3\\
  1 0 0 & \simeq & -4\\
  1 0 1 & \simeq & -3\\
  1 1 0 & \simeq & -2\\
  1 1 1 & \simeq & -1\\
\end{array}
$$

Using this convention, addition of signed integers can be done with
the standard addition of unsigned integers you just defined. For
multiplication, there is a slight twist but regular multiplication can
also be reused almost directly.

2. Fixed Point Representation: Some of the bits in the bitstring now
represent fractional numbers. Said otherwise, the least significant
bit does not represent the number $1$ but instead the number
$\frac1{2^k}$ for some fixed $k$ (thus "fixed point").

On $3$ bits with a fractional part of $1$ bit, the possible bitstring are then

$$
\begin{array}{lll}
  0 0 0 & \simeq & 0\\
  0 0 1 & \simeq & \frac12\\
  0 1 0 & \simeq & 1\\
  0 1 1 & \simeq & \frac32\\
  1 0 0 & \simeq & -2\\
  1 0 1 & \simeq & -\frac32\\
  1 1 0 & \simeq & -1\\
  1 1 1 & \simeq & -\frac12
\end{array}
$$

On 6 bits with a fractional part of $3$ bits, we have for instance

$$
\begin{array}{lll}
  0 ~~ 0 0  ~~ 0 0 0 & \simeq & 0\\
  0 ~~ 0 1  ~~ 0 0 1 & \simeq & 1 + \frac18\\
  0 ~~ 1 1  ~~ 0 0 0 & \simeq & 3\\
  1 ~~ 1 1  ~~ 1 1 1 & \simeq & -\frac18\\
  1 ~~ 1 1  ~~ 0 0 0 & \simeq & -1\\
  1 ~~ 1 0  ~~ 1 1 1 & \simeq & -1 - \frac18
\end{array}
$$


## 3.1 - Lifting operations to Signed FP reals (nothing to do)

This section defines for you arithmetic operations on signed FP reals:

In [ ]:
# Make a signed FP (Fixed Point) bitstring for real_nb

def makeSignedFP(n_frac, N, real_nb):
    int_nb = int(real_nb * (2 ** n_frac))
    if int_nb < 0:
        binary = nat2bl(N, (2 ** N) + (int_nb))
    else:
        binary = nat2bl(N, int_nb)
    return binary

for x in [0, 1+1/8, 3, -1/8, -1, -1-1/8]:
    print("%s corresponds to %f" % (str(makeSignedFP(3, 6, x)), x))



In [ ]:
# Opposite operation: get the real number from a bitstring

def readSignedFP(bitstring, n_frac):
    si = bl2nat(bitstring)
    sign = 1
    if len(bitstring) > 0 and bitstring[0] == 1:
        si = (2 ** (len(bitstring))) - si 
        sign = -1
    return sign * si/(2 ** n_frac)

print(readSignedFP([1,0,1], 1))

In [ ]:
# getValue, but for real numbers in FP notation

def getSignedFP(reg, val, n_frac):
    return readSignedFP(getValue(val,reg),n_frac)



In [ ]:
# Initialize a register in the state corresponding to a real in FP notation.

def constFPsigned(circ, n_frac, N, real_nb):
    binary = makeSignedFP(n_frac, N, real_nb)
    
    s = circ.allocn(N)
    for i in range(N):
        if binary[i] == 1:
            circ.x(s[i])
    return s

In [ ]:
# FP Signed Addition: nothing much to do beside forgetting the carry

def adderFPSigned(circ, n_frac, aa, bb):
     ss,c = adderV(circ,aa,bb)
     return ss



In [ ]:
# FP Signed Multiplication

def multFPSigned(circ, n_frac, aa, bb):
    na = len(aa)
    nb = len(bb)
    # signed extension
    aa_ext = circ.allocn(nb) + aa
    bb_ext = circ.allocn(na) + bb
    for i in range(nb):
        circ.cx(aa[0],aa_ext[i])
    for i in range(na):
        circ.cx(bb[0],bb_ext[i])
    # Multiplication (omit the carry -- overflow if too large)
    ss_ext = mult(circ, aa_ext, bb_ext)
    n_int = len(aa)-n_frac
    # Take relevant part
    return ss_ext[-na-nb:][n_int:len(aa)+n_int]




In the following cell, we set up a circuit computing

$$ x \mapsto 2.5 * x - 1.25 $$ 

and run it with $x = 3.5$. Do you read the correct value ?

In [ ]:

def exampleFPSigned():
    # 2 bits for the fractional part: enough as we won't go beyond 1/4
    # in precision
    n_frac = 2

    # 6 in total, leaving 3 for the integer part, enough for our
    # computation
    N = 6

    # Input register
    x = [i for i in range(N)] 

    # Set up a circuit
    circ = RevCirc(x)

    # Append all of the pieces, V-style
    coef0 = constFPsigned(circ, n_frac, N, -1.25)
    coef1 = constFPsigned(circ, n_frac, N, 2.5)
    monom = multFPSigned(circ, n_frac, coef1, x)
    result_reg = adderFPSigned(circ, n_frac, coef0, monom)

    # Define an input
    input = makeSignedFP(n_frac, N, 3.5)

    # Input valuation
    input_valuation = dict(zip(x, input))
    
    # Run the circuit
    valuation = circ.run(input_valuation)

    # print result!
    print("Resulting FP real: ", getSignedFP(result_reg, valuation, n_frac))

exampleFPSigned()    





## 3.2 - Polynomials

A polynomial is a function of the form

$$
x \mapsto 3\cdot x^2 - 2.5\cdot x + 1.125
$$

### 3.2.1 - Power function

One simple way to build a circuit for such a function is to define

$$
x \mapsto x^k
$$

for a fixed $k$.

**TODO** Fill-in the following code.

* `aa` is a register containing a FP signed real number
* `n` is a (regular, Python) positive integer

In [ ]:

def powerFPSigned(circ, n_frac, aa, n):
    assert(n > 0)

    ss = circ.allocn(len(aa)) # Placeholder for the result register,
                              # possibly to change

    ## TODO BEGIN


    ## TODO END
    
    return ss





Testing cell: you can play with `N`, `n_frac`, `p` and `input_x`.

Note: $2.5^3 = 15.625$, it should fit in `N=8` and `n_frac=3`.

In [ ]:

def testPowerFPSigned():
    
    # Shape of the register
    N = 8
    n_frac = 3

    # Power
    p = 3

    # value to power
    input_x = 2.5
    
    ## Do not change below
    
    # Input register
    x = [i for i in range(N)] 

    # Set up a circuit
    circ = RevCirc(x)

    # Power of x
    result_reg = powerFPSigned(circ, n_frac, x, p)

    # Define an input
    input_bl = makeSignedFP(n_frac, N, input_x)

    # Input valuation
    input_valuation = dict(zip(x, input_bl))
    
    # Run the circuit
    valuation = circ.run(input_valuation)

    # print result!
    print("Resulting FP real: ", getSignedFP(result_reg, valuation, n_frac))
    
testPowerFPSigned()





## 3.2.2 - Implementing Polynomials

You are now ready to implement a function such as

$$
x \mapsto 3\cdot x^2 - 2.5\cdot x + 1.125
$$

The coefficients of the above polynomial are coded as

`[3, -2.5, 1.125]`

Said otherwise, the coeff of the monomials of highest degree is at the
beginning.

**TODO** Fill-in the following code. The function returns a circuit
  realizing the polynomial.

In [ ]:

def polyCirc(coeffs, n_frac, N):
    inputs = [i for i in range(N)]
    circ = RevCirc(inputs)

    res = [] # placeholder for the register wires holding the
             # answer. Change accordingly
    
    ## TODO BEGIN


    ## TODO END
    
    return res, circ




Testing cell: you can play with `N`, `n_frac`, `p` and `input_x`.

In [ ]:

def testPoly():

    # Shape of registers
    N = 8
    n_frac = 3

    # coeffs of the polynomial
    coeffs = [3, -2.5, 1.125]

    # input value
    input_x = 1.5
    
    # Do not change: setting up the circuit and running it
    
    output, circ = polyCirc(coeffs, n_frac, N)
    
    a = makeSignedFP(n_frac,N, input_x)
    
    d = circ.run(dict(zip(circ.inputs, a)))
    
    print("The circuit says: ", getSignedFP(output, d, n_frac))

    exact_res = 0
    exp = 0
    for c in reversed(coeffs):
        exact_res = exact_res + c * (input_x ** exp)
        exp += 1
    
    print("The exact result should be", exact_res)
    

testPoly()




## 4 - Implementing the Inversion Operator of HHL

We are ready to approximate the inversion operator ! Following what we
already discussed, we shall focus on

$$ Inv : x \mapsto \arcsin\left(\frac1{16x}\right) $$

and we will use a Taylor expansion of the function.

### 4.1 - Taylor expansion of $Inv$ (nothing to do)

The function $Inv$ works on $[\frac1{16},1]$. We shall focus on the
Taylor expansion centered $x=0.5$.

The library `sympy` is our friend here: we can compute the successive
derivatives of arcsin algebraically

In [ ]:

# the function we care about
def Inv(x):
    return np.arcsin(1/(16 * x))

# The degree of the polynomial
degree = 3

# Point at which computing the Taylor polynomial
at_x = 0.5

# The Taylor polynomial !
xvar = sympy.symbols('x')

# At degree 0
poly = Inv(at_x)

# Degrees 1, 2, ...
deriv = sympy.functions.elementary.trigonometric.asin(1/(16*xvar))
for k in range(1,degree+1):
    deriv = sympy.diff(deriv, xvar)
    poly += deriv.subs(xvar,at_x)/math.factorial(k) * (xvar - at_x) ** k

def Inv_approx(x):
    return float(poly.subs(xvar,x))


print("The Taylor polynomial is")
print("   ", sympy.poly(poly))

# The list of coefficients using the convention we discussed when
# programming polynomials.
coeffs = sympy.poly(poly).all_coeffs()

print("The list of coefficients is")
print("   ", coeffs)




### 4.2 - The Circuit for the Taylor Approximation

**TODO**

1. Generate a circuit for the Taylor approximation computed in the
previous cell, using registers of size `N=16` with `n_frac=10` bits
for the fractional part. Note how the coefficients of the polynomial
are stored in `coeffs`, so you can reuse `polyCirc`.

2. Run it for several values between $0.1$ and $0.9$ (take, say, 50
points). Compare the number of bits of precision between
  * the Taylor expansion and the function computed by the circuit
  * The Taylor expansion and the "correct" $Inv$ function

You can use `-numpy.log2(abs(exact_value - approx_value))` to compute
how many bits two values differ from.

3. How does the error and the size of the circuit evolve with respect to
  * the size of the register?
  * the degree of the polynomial?